In [13]:
import uuid
from datetime import datetime

def get_data():
    import requests

    res = requests.get("https://randomuser.me/api/")
    res = res.json()
    res = res['results'][0]

    return res
    # print(json.dumps(res, indent=3))

def format_data(res):
    data = {}
    location = res['location']
    data['id'] = uuid.uuid4().int
    data['first_name'] = res['name']['first']
    data['last_name'] = res['name']['last']
    data['gender'] = res['gender']
    data['address'] = f"{str(location['street']['number'])} {location['street']['name']}, " \
                      f"{location['city']}, {location['state']}, {location['country']}"
    data['post_code'] = location['postcode']
    data['email'] = res['email']
    data['username'] = res['login']['username']
    data['dob'] = res['dob']['date']
    data['registered_date'] = res['registered']['date']
    data['phone'] = res['phone']
    data['picture'] = res['picture']['medium']
    return data


In [ ]:
import json
from kafka import KafkaProducer
import time
import logging

# producer = KafkaProducer(bootstrap_servers=['broker:29092'], max_block_ms=5000)
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], max_block_ms=5000)
curr_time = time.time()

try:
    res = get_data()
    res = format_data(res)
    print(res)

    producer.send('users_created', json.dumps(res).encode('utf-8'))
except Exception as e:
        logging.error(f'An error occured: {e}')

{'id': 221982996184399843825839779951673967176, 'first_name': 'Bob', 'last_name': 'May', 'gender': 'male', 'address': '2190 St. John’S Road, Kingston upon Hull, Northamptonshire, United Kingdom', 'post_code': 'SF16 7JH', 'email': 'bob.may@example.com', 'username': 'blackostrich542', 'dob': '1997-07-11T19:11:27.521Z', 'registered_date': '2017-05-01T10:31:20.540Z', 'phone': '015395 76202', 'picture': 'https://randomuser.me/api/portraits/med/men/19.jpg'}


ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Closing connection. KafkaConnectionError: Socket EVENT_READ without in-flight-requests
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv4 ('127.0.0.1', 9092)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv4 ('127.0.0.1', 9092)]>: Closing connection. KafkaConnectionError: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Closing connection. KafkaConnectionError: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv4 ('127.0.0.1', 9092)]>: socket disconnected
ERROR:kafka.conn:<BrokerConnection node_id=1 host=localhost:9092 <connected> [IPv4 ('127.0.0.